In [10]:
import pandas as pd
import numpy as np

In [11]:
data_folder = '/home/kate/Research/Property/Data/'

In [12]:
data = pd.read_csv(data_folder+'property_water_claims_non_cat_fs_v5.csv', error_bad_lines=False, index_col=False)

For better fit Normal distribution remove only small losses

In [13]:
dataset = data[data.cova_il_nc_water >=100].copy()

In [14]:
featureset_predictors  = [
'cova_deductible',
'roofcd_encd',
'water_risk_sev_3_blk',
'sqft',
'rep_cost_3_blk',
'yearbuilt',
'ecy',
'usagetype_encd',    
'log_yearbuilt',
'log_sqft',
'log_water_risk_sev_3_blk',
'log_ecy'
]

In [15]:
target_column = 'log_cova_il_nc_water' 

In [16]:
from sklearn.model_selection import KFold
kfold =5 
kf = KFold(n_splits=kfold, random_state=42, shuffle=True)

In [17]:
folds = list()
for i in range(0,kfold):
    dataset['fold_%s'%i]=0
    folds.append('fold_%s'%i)

In [18]:
#for year in dataset.cal_year.unique():
for year in [2019]:
    print('Creating training dataset for %s'%year)
    training_dataset=dataset[(dataset.cal_year != year) & (dataset.cal_year != 2020) ][featureset_predictors + folds+ [target_column]] 
    print('Size = %s'%len(training_dataset))
    #aggregation does not impact because the loss is very unique and is not repeated at all
    #but it can be impartant for other type of losses
    #training_dataset = pd.DataFrame({'cova_ic_nc_water' : training_dataset.groupby(featureset_predictors + [target_column])[aggregated_column].sum()}).reset_index()
    print('Size of unique values = %s'%len(training_dataset))
    print('Creating folds')
    for i, (train_index, test_index) in enumerate(kf.split(training_dataset[featureset_predictors], training_dataset[target_column])):
        print(' fold: {}  of  {} : '.format(i+1, kfold))
        training_dataset['fold_%s'%i]=0
        training_dataset.iloc[train_index,training_dataset.columns.get_loc('fold_%s'%i)]=1
        print('Size = %s'%sum(training_dataset['fold_%s'%i]))
    print('Saving training dataset for %s'%year)
    training_dataset.to_csv('%sproperty_wcs_training_for_normal.csv'%(data_folder),header=True,index=False)
        

Creating training dataset for 2019
Size = 12001
Size of unique values = 12001
Creating folds
 fold: 1  of  5 : 
Size = 9600
 fold: 2  of  5 : 
Size = 9601
 fold: 3  of  5 : 
Size = 9601
 fold: 4  of  5 : 
Size = 9601
 fold: 5  of  5 : 
Size = 9601
Saving training dataset for 2019
